In [1]:
import pandas as pd
import numpy as np
import folium

In [17]:
grant_data = pd.read_csv('P3_GrantExport.csv', sep=';',
                         usecols = ['Institution','University', 
                                    'Approved Amount'])
grant_data.head()

,Institution,University,Approved Amount
0,NaN,Nicht zuteilbar - NA,11619.00
1,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,41022.00
2,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",79732.00
3,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,52627.00
4,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",120042.00


In [15]:
grant_data_cleaned = grant_data[(grant_data['Approved Amount']!='data not included in P3') & 
                                (grant_data['University']!='Nicht zuteilbar - NA') &
                                (grant_data['University']!='NPO (Biblioth., Museen, Verwalt.) - NPO') &
                                (grant_data['University']!='Firmen/Privatwirtschaft - FP') &
                                grant_data['University'].notnull()]
grant_data_cleaned['Approved Amount'] = grant_data_cleaned['Approved Amount'].astype(float)

/Users/Artem_mac/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
all_money = grant_data_cleaned['Approved Amount'].sum()
data_by_university = grant_data_cleaned.groupby('University').sum().sort_values('Approved Amount', ascending=False)
data_by_university.head()

,Approved Amount
University,
Université de Genève - GE,1.838237e+09
Universität Zürich - ZH,1.826843e+09
ETH Zürich - ETHZ,1.635597e+09
Universität Bern - BE,1.519373e+09
Universität Basel - BS,1.352251e+09


In [20]:
sum = 0
for index, row in data_by_university.iterrows():
    sum += row['Approved Amount']
    print(index)
    if sum >= 0.95*all_money:
        break

Université de Genève - GE
Universität Zürich - ZH
ETH Zürich - ETHZ
Universität Bern - BE
Universität Basel - BS
Université de Lausanne - LA
EPF Lausanne - EPFL
Université de Fribourg - FR
Université de Neuchâtel - NE
Paul Scherrer Institut - PSI
Università della Svizzera italiana - USI
Eidg. Anstalt für Wasserversorgung - EAWAG
Universität St. Gallen - SG
Eidg. Material und Prüfungsanstalt - EMPA


In [21]:
cantons = pd.Series({"Université de Genève - GE": "Genève", 
                     "Universität Zürich - ZH": "Zürich", 
                     "ETH Zürich - ETHZ": "Zürich", 
                     "Universität Bern - BE": "Bern/Berne", 
                     "Universität Basel - BS": "Basel-Stadt", 
                     "Université de Lausanne - LA": "Vaud", 
                     "EPF Lausanne - EPFL": "Vaud", 
                     "Université de Fribourg - FR": "Fribourg", 
                     "Université de Neuchâtel - NE": "Neuchâtel", 
                     "Paul Scherrer Institut - PSI": "Aargau", 
                     "Università della Svizzera italiana - USI": "Ticino", 
                     "Eidg. Anstalt für Wasserversorgung - EAWAG": "Zürich", 
                     "Universität St. Gallen - SG": "St. Gallen", 
                     "Eidg. Material und Prüfungsanstalt - EMPA": "Zürich"})

In [24]:
data_by_university['Canton'] = cantons
data_by_university.dropna(inplace = True)

,Approved Amount,Canton
University,,
Université de Genève - GE,1.838237e+09,Genève
Universität Zürich - ZH,1.826843e+09,Zürich
ETH Zürich - ETHZ,1.635597e+09,Zürich
Universität Bern - BE,1.519373e+09,Bern/Berne
Universität Basel - BS,1.352251e+09,Basel-Stadt
Université de Lausanne - LA,1.183291e+09,Vaud
EPF Lausanne - EPFL,1.175316e+09,Vaud
Université de Fribourg - FR,4.575262e+08,Fribourg
Université de Neuchâtel - NE,3.832046e+08,Neuchâtel


In [25]:
data_by_canton = data_by_university.groupby('Canton').sum().sort_values('Approved Amount', ascending=False)
data_by_canton.reset_index(inplace = True)

In [26]:
data_by_canton

,Canton,Approved Amount
0,Zürich,3.595633e+09
1,Vaud,2.358607e+09
2,Genève,1.838237e+09
3,Bern/Berne,1.519373e+09
4,Basel-Stadt,1.352251e+09
5,Fribourg,4.575262e+08
6,Neuchâtel,3.832046e+08
7,Aargau,1.152690e+08
8,Ticino,8.497055e+07
9,St. Gallen,6.919495e+07


In [141]:
map = folium.Map(location=[48, -102], zoom_start=3)
map.choropleth(geo_path="ch-cantons.topojson.json", data=data_by_canton,
             columns=['Canton', 'Approved Amount'],
             key_on='geometries.id',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Unemployment Rate (%)')
map.save('map.html')

/Users/Artem_mac/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: 'threshold_scale' default behavior has changed. Now you get a linear scale between the 'min' and the 'max' of your data. To get former behavior, use folium.utilities.split_six.


AttributeError: 'NoneType' object has no attribute 'get'